<a href="https://colab.research.google.com/github/sahug/ds-tensorflow-colab/blob/master/Tensorflow%20-%20Tune%20Hyperparameter%20with%20Keras%20Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tensorflow - Tune Hyperparameter with Keras Tuner**

**Setup**

In [34]:
import tensorflow as tf
from tensorflow import keras

In [31]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 8.4 MB/s 


In [32]:
import keras_tuner as kt

**Download and prepare the dataset**

In [35]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [36]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

**Define the model**

When you build a model for hypertuning, you also define the hyperparameter search space in addition to the model architecture. The model you set up for hypertuning is called a hypermodel.

You can define a hypermodel through two approaches:

- By using a model builder function
- By subclassing the HyperModel class of the Keras Tuner API

In [37]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

**Instantiate the tuner and perform hypertuning**

In [38]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

Create a **callback** to stop training early after reaching a certain value for the validation loss.

In [39]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the **hyperparameter search**. The arguments for the search method are the same as those used for **tf.keras.model.fit in addition to the callback above**.

In [40]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 15s]
val_accuracy: 0.8815000057220459

Best val_accuracy So Far: 0.8866666555404663
Total elapsed time: 00h 14m 28s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit



The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



**Train the model**

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [41]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.4945 - accuracy: 0.8238 - val_loss: 0.4002 - val_accuracy: 0.8542
Epoch 2/50
1500/1500 [==============================] - 9s 6ms/step - loss: 0.3695 - accuracy: 0.8650 - val_loss: 0.3603 - val_accuracy: 0.8709
Epoch 3/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3336 - accuracy: 0.8762 - val_loss: 0.3395 - val_accuracy: 0.8773
Epoch 4/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3049 - accuracy: 0.8871 - val_loss: 0.3235 - val_accuracy: 0.8835
Epoch 5/50
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2893 - accuracy: 0.8930 - val_loss: 0.3261 - val_accuracy: 0.8804
Epoch 6/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2731 - accuracy: 0.8985 - val_loss: 0.3288 - val_accuracy: 0.8830
Epoch 7/50
1500/1500 [==============================] - 7s 5ms/step - loss: 0.2576 - accuracy: 0.9034 - val_loss: 0.3267 - val_accuracy:

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [42]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/28
1500/1500 [==============================] - 8s 5ms/step - loss: 0.4932 - accuracy: 0.8256 - val_loss: 0.4031 - val_accuracy: 0.8518
Epoch 2/28
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3667 - accuracy: 0.8661 - val_loss: 0.3797 - val_accuracy: 0.8602
Epoch 3/28
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3297 - accuracy: 0.8780 - val_loss: 0.3356 - val_accuracy: 0.8792
Epoch 4/28
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3031 - accuracy: 0.8879 - val_loss: 0.3502 - val_accuracy: 0.8735
Epoch 5/28
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2829 - accuracy: 0.8945 - val_loss: 0.3249 - val_accuracy: 0.8869
Epoch 6/28
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2696 - accuracy: 0.8984 - val_loss: 0.3127 - val_accuracy: 0.8891
Epoch 7/28
1500/1500 [==============================] - 8s 6ms/step - loss: 0.2561 - accuracy: 0.9047 - val_loss: 0.3375 - val_accuracy:

To finish this tutorial, evaluate the hypermodel on the test data.

In [43]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 3ms/step - loss: 0.4478 - accuracy: 0.8866
[test loss, test accuracy]: [0.4478291869163513, 0.8866000175476074]


The **my_dir/intro_to_kt** directory contains detailed logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional **overwrite=True** argument while instantiating the tuner.